# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects

In [2]:
import datetime
import os
import sys

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole, ListSortOrder
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 14-Jun-2025 10:07:50


## 1. Grounding Bing Search

### Connect to AI Foundry

In [5]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to bing grounding service

In [6]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

### Create agent with Bing Grounding tool

In [7]:
# Create the agent with the Bing grounding tool
instructions = "You are an AI Agent that can do some web search using Bing"
agent = project_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-bing-grounding-agent",
        instructions=instructions,
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")

bing
bing.definitions


Created agent, ID: asst_tMcavEj0LE5740DP2Hg56u6s


[{'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject/connections/aqbinggrounding002', 'market': '', 'set_lang': '', 'count': 5, 'freshness': ''}]}}]

### Create THREAD

In [8]:
# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_UXYPGOvTJFkVn831XSmYRLoM


### Example #1

In [9]:
query = "What is the current price of MICROSOFT stock?"

### Create first message in the thread

In [10]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_OCXvQSsPdtmk2nSsj8NhWgn8


### Create first RUN

In [11]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


### Check status of the RUN and tool calls

In [12]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage


Step step_OEEdl6kPvxk0Hkd6TnbiJ3Nr status: completed
Step step_1cIbfmSF5SVrRa2mR1AY2Sdb status: completed
  Tool calls:
    Tool Call ID: call_4RNO76SikMVT6cZX62f8vIPK
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "current price of Microsoft stock")



{'prompt_tokens': 2810, 'completion_tokens': 67, 'total_tokens': 2877, 'prompt_token_details': {'cached_tokens': 0}}

### Get response

In [13]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: As of the most recent market close, the price of Microsoft (MSFT) stock is $474.96 USD. Please note that prices may fluctuate during after-hours trading and when markets are open【3:4†source】.
URL Citation: [Microsoft Stock Price Today | NASDAQ: MSFT Live - Investing.com](https://www.investing.com/equities/microsoft-corp)


### Example #2

In [14]:
query = "find out about the latest iPhone"

In [15]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_B2CzEUiWXHYPozmNw4FV3rJZ


In [16]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [17]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_O6YXOWSdUszRJV8nlhj7np2x status: completed
Step step_gyCqRDMW0oh1WRmXLjYwrcxb status: completed
  Tool calls:
    Tool Call ID: call_XbdCdcUV1RslB4GmY7mq6F3a
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "latest iPhone 2025")



{'prompt_tokens': 4257, 'completion_tokens': 348, 'total_tokens': 4605, 'prompt_token_details': {'cached_tokens': 0}}

In [18]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: The latest iPhone in 2025 is centered around the upcoming iPhone 17 series, which is expected to debut around September 2025. The 2025 lineup includes multiple models, most notably the iPhone 17, iPhone 17 Pro, and a brand-new, ultra-slim model called the iPhone 17 Air. Key upgrades for the newest iPhones include:

- A new, lighter design with titanium for flagship models and a slimmer profile for the iPhone 17 Air.
- Major camera improvements, such as a periscope lens for up to 10x optical zoom on higher-end models, and enhanced computational photography features.
- The latest A-series chips across all models for improved performance and energy efficiency.
- Better battery life and faster charging.
- Design differentiation between entry-level and flagship models, including ProMotion displays on the premium versions.
- The entry-level iPhone SE is also getting a redesign, adopting an all-screen look and updated camera tech.

These updates aim to modernize the iPhone wit

### Example #3

In [19]:
query = "What is the weather informations for Mexico city? Show the report and use emojis and be funny"

In [20]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_8NP2Yzx6OIKoCsZc17KLRJ2Q


In [21]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [22]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_ArYuHpPqbeCCGU1YN5dUg5Pn status: completed
Step step_UGNccMTD8jHVkt5BrKD47eNy status: completed
  Tool calls:
    Tool Call ID: call_AigtPWqYMDBrXq2uDzQxavXV
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "current weather Mexico City")



{'prompt_tokens': 5428, 'completion_tokens': 283, 'total_tokens': 5711, 'prompt_token_details': {'cached_tokens': 0}}

In [23]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: 🌦️ Hola from Mexico City, amigos! Here’s your sparkly weather report, now with 20% more emojis and 100% more laughs!

- Current Temperature: 22°C (that’s 72°F for those who like to keep things spicy 🔥).
- Weather: Cloudy, like a tortilla that dropped in the flour bucket. ☁️
- Showers: Expect some wet action late in the afternoon. Take an umbrella or risk the “wet dog” look! ☔🐶
- Humidity: 52%—just enough to keep your hair interesting! 💇‍♂️💧
- Wind: Breezy from the northeast at 12 mph, so hold onto your taco hats! 🎩🌮
- UV Index: 0 right now, so no sunburn excuses if you forget sunscreen! 🧴🌞

Tonight will be partly cloudy with a chance of sprinkles—think “donut” sprinkles, but less delicious. 🍩🌧️

Stay dry, wear those stylish rain boots, and if you see the sun, take a selfie—it's rare! 📸😎

¡Disfruta el clima!【11:0†Mexico City, México City, Mexico Current Weather | AccuWeather】
URL Citation: [Mexico City, México City, Mexico Current Weather | AccuWeather](https://www.accuw

### Post processing

In [24]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_tMcavEj0LE5740DP2Hg56u6s, Name: my-bing-grounding-agent, Model: gpt-4.1, Instructions: You are an AI Agent that can do some web search using Bing


In [26]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")

In [27]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")